In [1]:
import pandas as pd
import numpy as np
from pandas import datetime
from matplotlib import pyplot as plt

C:\Users\gfgf9\AppData\Local\Temp/ipykernel_30248/1649430976.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [55]:
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv'

df = pd.read_csv(input_file,
                 index_col=[0],
                 parse_dates=[0],
                 date_parser=parser)

In [56]:
#visualization setup
%matplotlib
from matplotlib import pyplot as plt
import seaborn; seaborn.set()  # set plot styles
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = [10, 5]
plt.ion() # enable the interactive mode

Using matplotlib backend: Qt5Agg


In [57]:
#visulaize the 'CO(GT)' variable
df['CO(GT)'].plot()

<AxesSubplot:xlabel='Datetime'>

In [58]:
#linear interpolation
co=df['CO(GT)'].copy()
co.interpolate(inplace=True)

In [59]:
# Visualize original and imputed data
plt.plot(df['CO(GT)'], label='original', zorder=2)
plt.plot(co, label='linear interpolation', zorder=1)
plt.legend(loc='best')

In [60]:
plt.boxplot(co)
plt.title("Detecting outliers using Boxplot")
plt.xlabel('CO(GT)')

Text(0.5, 0, 'CO(GT)')

In [61]:
corr_matrix = df.corr()
print(corr_matrix)

                 CO(GT)  PT08.S1(CO)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)  \
CO(GT)         1.000000     0.877203       0.914973  0.792557     -0.701703   
PT08.S1(CO)    0.877203     1.000000       0.892964  0.713654     -0.771938   
PT08.S2(NMHC)  0.914973     0.892964       1.000000  0.704435     -0.796703   
NOx(GT)        0.792557     0.713654       0.704435  1.000000     -0.655707   
PT08.S3(NOx)  -0.701703    -0.771938      -0.796703 -0.655707      1.000000   
NO2(GT)        0.679262     0.641529       0.646245  0.763111     -0.652083   
PT08.S4(NO2)   0.639470     0.682881       0.777254  0.233731     -0.538468   
PT08.S5(O3)    0.851403     0.899324       0.880578  0.787046     -0.796569   
RH             0.040218     0.114606      -0.090380  0.221032     -0.056740   
AH             0.065809     0.135324       0.186933 -0.149323     -0.232017   
C6H6(GT)       0.845144     0.883795       0.981950  0.626638     -0.735744   

                NO2(GT)  PT08.S4(NO2)  PT08.S5(O3) 

In [62]:
rh = df['RH'].copy().interpolate() # RH(Relative Humidity)

In [63]:
plt.scatter(co,rh,s=12,c='black')
plt.xlabel('CO(GT)')
plt.ylabel('RH')

Text(0, 0.5, 'RH')

In [64]:
# Choose the most correlated variable
nmhc = df['PT08.S2(NMHC)'].copy().interpolate() # NMHC(non-methane hydrocarbons)

In [65]:
plt.scatter(co,nmhc,s=12,c='black')
plt.xlabel('CO(GT)')
plt.ylabel('PT08.S2(NMHC)')

Text(0, 0.5, 'PT08.S2(NMHC)')

In [66]:
q1 = co.quantile(0.25)
median = co.quantile(0.5)
q3 = co.quantile(0.75)
print(q1, median, q3)

1.1 1.8 2.9


In [67]:
iqr= q3-q1
upper_fence=q3+1.5*iqr
lower_fence =q1 - 1.5*iqr
print(upper_fence,lower_fence)

5.6 -1.5999999999999996


In [68]:
outliers= co.loc[(co>upper_fence)| (co<0)]
print(outliers)

Datetime
2004-03-11 19:00:00    6.9
2004-03-11 20:00:00    6.1
2004-03-12 20:00:00    6.6
2004-03-14 20:00:00    5.9
2004-03-15 09:00:00    8.1
                      ... 
2005-03-23 19:00:00    6.2
2005-03-23 20:00:00    7.2
2005-03-24 19:00:00    5.9
2005-03-24 20:00:00    7.5
2005-03-25 19:00:00    5.7
Name: CO(GT), Length: 224, dtype: float64


In [69]:
mask = co.index.isin(outliers.index)
mask

array([False, False, False, ..., False, False, False])

In [70]:
plt.plot(co[~mask], label='normal', color='blue',
    marker='o', markersize=3, linestyle='None')
plt.plot(outliers, label='outliers', color='red',
    marker='x', markersize=3, linestyle='None')
plt.legend(loc='best')

In [71]:
co_refined = co.copy()
co_refined[mask] = np.nan
print(co_refined[mask])
co_refined.plot()

Datetime
2004-03-11 19:00:00   NaN
2004-03-11 20:00:00   NaN
2004-03-12 20:00:00   NaN
2004-03-14 20:00:00   NaN
2004-03-15 09:00:00   NaN
                       ..
2005-03-23 19:00:00   NaN
2005-03-23 20:00:00   NaN
2005-03-24 19:00:00   NaN
2005-03-24 20:00:00   NaN
2005-03-25 19:00:00   NaN
Name: CO(GT), Length: 224, dtype: float64


<AxesSubplot:xlabel='Datetime'>

In [41]:
co_refined.interpolate(inplace=True)
co_refined.plot()

<AxesSubplot:xlabel='Datetime'>

In [42]:
"""
Detecting Outliers with Z-Scores
"""
import seaborn as sns
sns.distplot(co)

c:\users\gfgf9\appdata\local\programs\python\python38\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:xlabel='CO(GT)', ylabel='Density'>

In [43]:
mean = np.mean(co)
std = np.std(co)
print(mean, std)

2.126146200705354 1.4369815748596468


In [44]:

# Calculate Z-scores for each data points

outliers = []
thres = 3   # Z-score threshold

for i in co:
    z_score = (i-mean) / std
    if (np.abs(z_score) > thres):
        print(z_score)
        outliers.append(i)

3.3221398818289782
3.1133689377552507
4.157223658123887
4.087633343432645
3.0437786230640085
3.1133689377552507
3.809272084667675
3.1829592524464934
3.1133689377552507
3.600501140593948
3.5309108259027058
3.0437786230640085
3.0437786230640085
3.1829592524464934
3.0437786230640085
3.600501140593948
3.1829592524464934
3.739681769976433
3.0437786230640085
3.1829592524464934
3.3917301965202205
4.087633343432645
3.8788623993589177
3.2525495671377356
4.365994602197615
3.2525495671377356
4.018043028741403
4.365994602197615
4.087633343432645
4.365994602197615
5.131488063801282
3.0437786230640085
3.1133689377552507
3.739681769976433
3.2525495671377356
4.018043028741403
4.087633343432645
4.922717119727554
3.3917301965202205
3.7860753131039275
4.180420429687635
4.574765546271342
4.157223658123887
4.087633343432645
3.1133689377552507
3.2525495671377356
3.94845271405016
5.618620266639979
3.1133689377552507
3.739681769976433
3.5309108259027058
4.296404287506373
4.5051752315800995
3.3221398818289782


In [45]:
# Simplified version of filtering outliers
outliers = co.loc[np.abs((co-mean)/std) > 3].copy()
outliers

Datetime
2004-03-11 19:00:00    6.9
2004-03-12 20:00:00    6.6
2004-03-15 09:00:00    8.1
2004-03-15 19:00:00    8.0
2004-03-15 20:00:00    6.5
                      ... 
2005-03-14 19:00:00    6.5
2005-03-14 20:00:00    6.5
2005-03-15 20:00:00    7.4
2005-03-23 20:00:00    7.2
2005-03-24 20:00:00    7.5
Name: CO(GT), Length: 116, dtype: float64

In [47]:
mask = co.index.isin(outliers.index)
mask

array([False, False, False, ..., False, False, False])

In [49]:
sns.distplot(co,axlabel = 'CO(GT)', label = 'original')
sns.distplot(co[~mask], label = 'outliers removed')
plt.legend(loc = 'best')

c:\users\gfgf9\appdata\local\programs\python\python38\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
c:\users\gfgf9\appdata\local\programs\python\python38\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [51]:
floor = co.quantile(0.1)
cap= co.quantile(0.9)

In [52]:
co.loc[co < floor] = floor
co.loc[co > cap] = cap

In [53]:
co.plot()

<AxesSubplot:xlabel='Datetime'>